In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Sales_order_Query='''


with original
as 
(SELECT 
min(dly_ord_itm_actvy_ts) as first_add,
bus_prod_instnc_id
 FROM `cio-datahub-enterprise-pr-183a.ent_cust_ord_actvy.bq_dly_wln_ord_item_actvy` WHERE ord_act_typ_cd = 'Add' and prod_typ_cd in ('SMHM') and ord_act_stat_cd = 'Processed' and bus_prod_instnc_id is not null
 and date(dly_ord_itm_actvy_ts) >= '2023-10-01' and date(dly_ord_itm_actvy_ts) < '2024-01-01'

 group by bus_prod_instnc_id) /* Fetch original order date for the product instance */

,details as
(
SELECT 
dly_ord_itm_actvy_ts,
bus_billg_acct_num,
bus_prod_instnc_id,
prod_nm,
munic_nm,
prov_state_cd,
bi_chnl_tag_cd,
chnl_org_id,
prod_typ_cd,
src_sls_rep_cd,
src_typ_cd,
src_usr_chnl_txt,
txn_sub_typ_txt,
SLS_ACTVY_TXT
 FROM `cio-datahub-enterprise-pr-183a.ent_cust_ord_actvy.bq_dly_wln_ord_item_actvy` WHERE ord_act_typ_cd = 'Add' 
 --and prod_typ_cd in ('SMHM','HSIC','SING','TTV') 
 and prod_typ_cd in ('SMHM')
 and ord_act_stat_cd = 'Processed'
 and date(dly_ord_itm_actvy_ts) >= '2023-10-01' and date(dly_ord_itm_actvy_ts) < '2024-01-01' 
 
 )
 
 
 
  /* Gather channel details */

,channel_refs as (


with
latest_update as (
SELECT
chnl_org_id,
max(chnl_org_key) as latest_key FROM `cio-datahub-enterprise-pr-183a.ent_sls_chnl.bq_channel_org_dim`
group by chnl_org_id

)

select distinct
t1.chnl_org_id,
chnl_org_txt from `cio-datahub-enterprise-pr-183a.ent_sls_chnl.bq_channel_org_dim` t1 inner join latest_update on latest_key = chnl_org_key

)

select
date(details.dly_ord_itm_actvy_ts) as Order_date,
details.bus_billg_acct_num as BAN,
details.bus_prod_instnc_id as Prod_Instnc_ID,
details.prod_nm as Prod_nm,
details.munic_nm,
details.prov_state_cd,
details.bi_chnl_tag_cd as Sales_Channel_tag,
details.chnl_org_id,
channel_refs.chnl_org_txt,
details.prod_typ_cd as Prod_Type,
details.src_sls_rep_cd as Sales_Agent_ID,
details.src_typ_cd,
details.src_usr_chnl_txt,
details.txn_sub_typ_txt,
details.SLS_ACTVY_TXT
 from details inner join original on original.bus_prod_instnc_id = details.bus_prod_instnc_id and original.first_add = details.dly_ord_itm_actvy_ts  /* Inner join them to prevent duplicates */
 left join channel_refs on details.chnl_org_id = channel_refs.chnl_org_id /* left joining to get channel dealer names */
 where bi_chnl_tag_cd <> 'UNKNOWN' 
 order by details.dly_ord_itm_actvy_ts

'''

In [ ]:
DF_sales_Order=extract_bq_data(bq_client, sql=Sales_order_Query)

In [ ]:
DF_sales_Order.info()

In [ ]:
DF_sales_Order.head()

In [ ]:
DF_sales_Order['Prod_Type'].value_counts()

In [ ]:
DF_sales_Order.to_csv('SMHM_Sales_order_Oct_Dec_2023.csv',index=False)

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Your project ID
project_id = 'bi-srv-hsmsd-3c-pr-ca2cd4'
# Your dataset name
dataset_name = 'hsmsd_3c_rpt_dataset'
# Your table name
table_name = 'bq_rpt_chnl_order_ffh_dtl'

# Initialize a BigQuery client
bq_client
# Construct the fully-qualified table ID
table_id = f"{project_id}.{dataset_name}.{table_name}"

# Get the table
table = bq_client.get_table(table_id)

# Prepare lists to hold schema information
column_names = []
column_types = []
column_descriptions = []

# Extract schema details
for schema_field in table.schema:
    column_names.append(schema_field.name)
    column_types.append(schema_field.field_type)
    column_descriptions.append(schema_field.description or 'No description')

# Create a pandas DataFrame
schema_df = pd.DataFrame({
    'Column Name': column_names,
    'Data Type': column_types,
    'Description': column_descriptions
})

# Display the DataFrame
print(schema_df)

In [ ]:
schema_df.to_csv('bq_rpt_chnl_order_ffh_dtl_Schema.csv',index=False)